# Cosine similarity

In [1]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn

In [2]:
def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'left_side': left_side,
                         'right_side': right_side,
                         'similarity': similarity})

In [3]:
!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


print(ngrams('Klazenga, N.'))
print(ngrams('Beauglehole, A.C.'))


['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
['Bea', 'eau', 'aug', 'ugl', 'gle', 'leh', 'eho', 'hol', 'ole', 'le ', 'e A', ' A ', 'A C']


In [4]:
# Create Wikidata items data frame
df_wikidata = pd.read_csv('data/wikidata_persons.csv')
df_wikidata = df_wikidata.iloc[:, 1:]

df_wikidata.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,11,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,item,itemLabel,surname,initials,canonical_string,orcid,viaf,isni,harv,ipni,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
0,http://www.wikidata.org/entity/Q67329,Carl Gustav Jablonsky,Jablonsky,C.G.,"Jablonsky, C.G.",NaN,24944037,0000 0000 5526 3001,NaN,NaN,...,1787.0,NaN,NaN,http://www.wikidata.org/wiki/Q67329,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q68738,Hermann Müller,Müller,H.,"Müller, H.",NaN,2532803,0000 0001 0837 1728,36129,6735-1,...,1883.0,NaN,NaN,http://www.wikidata.org/wiki/Q68738,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6735-1,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q66902,Hermann Lebert,Lebert,H.,"Lebert, H.",NaN,27833384,0000 0001 2100 1924,67704,22162-1,...,1878.0,NaN,NaN,http://www.wikidata.org/wiki/Q66902,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/22162-1,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q69552,Carl Julius Bernhard Börner,Börner,C.J.B.,"Börner, C.J.B.",NaN,118457204,0000 0001 1085 9124,1763,12350-1,...,1953.0,NaN,NaN,http://www.wikidata.org/wiki/Q69552,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12350-1,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q66379,Ludwig Bechstein,Bechstein,L.,"Bechstein, L.",NaN,36914329,0000 0001 2278 3438,NaN,NaN,...,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q66379,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Create data frame with unique canonical strings
df_wikidata_unique = df_wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
cols = df_wikidata_unique.columns.tolist()

df_wikidata_unique

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",2
1,"(1), G.Z.",1
2,"(Bert), A.A.H.",1
3,"(Entomologist), C.K.",1
4,"(Gilyazetdinova), D.G.",1
...,...,...
107831,"Ǒwaki, K.",1
107832,"Șerbanescu, I.",1
107833,"Șuster, P.M.",1


In [6]:
# Vectorize Wikidata name (use fit_transform())
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(df_wikidata_unique['canonical_string'])

In [7]:
# Load AVH collectors data set
df_avh = pd.read_csv('data/avh_collectors.csv')
df_avh = df_avh.iloc[:, 1:]
df_avh.drop(columns=['i18nCode', 'fq'], inplace=True)

df_avh.head()

,label,count,start_date,end_date,activity_span
0,"Beauglehole, A.C.",90942,1865.0,2005.0,140.0
1,"Forster, P.I.",64649,1955.0,2018.0,63.0
2,"Hyland, B.",57265,1952.0,2008.0,56.0
3,"Latz, P.",51230,1875.0,2019.0,144.0
4,"Streimann, H.",45346,1896.0,2001.0,105.0


In [8]:
# Vectorize AVH collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_avh['label'])

# Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), ntop=1, lower_bound=0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Create the data frame
matches_df = get_matches_df(matches, df_avh['label'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.13402223587036133


,left_side,right_side,similarity
0,"Beauglehole, A.C.","Beauglehole, A.C.",1.000
1,"Forster, P.I.","Forster, P.I.",1.000
2,"Hyland, B.","Hyland, B.",1.000
3,"Latz, P.","Latz, P.K.",0.848
4,"Streimann, H.","Streimann, H.",1.000


In [9]:
# Join matches data frame to AVH collectors data frame
df_avh_matches = pd.merge(df_avh, matches_df, left_on='label', right_on='left_side', how='left')

df_avh_matches

# Join Wikidata items
df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_avh_wikidata_cossim = df_avh_matches_wikidata[['label', 'count', 'start_date', 'end_date', 'activity_span', 
                                                  'left_side', 'right_side', 'similarity', 'dup_count', 
                                                  'item', 'itemLabel', 'surname', 'initials', 'canonical_string', 
                                                  'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                  'enc_au_sc_id', 'yob', 'yod', 'wyb', 'wye', 
                                                  'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 
                                                  'bloodhound_link', 'enc_au_sc_link', 'au_dict_bio_link']]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_avh_wikidata_cossim.sort_values(by=['similarity', 'dup_count', 'count'], ascending=[False, True, False], inplace=True)

df_avh_wikidata_cossim


/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,count,start_date,end_date,activity_span,left_side,right_side,similarity,dup_count,item,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
0,"Beauglehole, A.C.",90942,1865.0,2005.0,140.0,"Beauglehole, A.C.","Beauglehole, A.C.",1.0,1.0,http://www.wikidata.org/entity/Q16744919,...,2002.0,NaN,NaN,http://www.wikidata.org/wiki/Q16744919,NaN,NaN,NaN,https://bloodhound-tracker.net/Q16744919,http://www.eoas.info/biogs/P000214b,NaN
1,"Forster, P.I.",64649,1955.0,2018.0,63.0,"Forster, P.I.","Forster, P.I.",1.0,1.0,http://www.wikidata.org/entity/Q9057027,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q9057027,NaN,NaN,https://www.ipni.org/a/18907-1,NaN,NaN,NaN
2,"Hyland, B.",57265,1952.0,2008.0,56.0,"Hyland, B.","Hyland, B.",1.0,1.0,http://www.wikidata.org/entity/Q4893242,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q4893242,NaN,NaN,https://www.ipni.org/a/4262-1,NaN,NaN,NaN
4,"Streimann, H.",45346,1896.0,2001.0,105.0,"Streimann, H.","Streimann, H.",1.0,1.0,http://www.wikidata.org/entity/Q21339679,...,2001.0,NaN,NaN,http://www.wikidata.org/wiki/Q21339679,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/15669-1,NaN,http://www.eoas.info/biogs/P005606b,NaN
5,"Elix, J.A.",39702,1878.0,2020.0,142.0,"Elix, J.A.","Elix, J.A.",1.0,1.0,http://www.wikidata.org/entity/Q21339171,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21339171,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/18445-1,NaN,http://www.eoas.info/biogs/P005725b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,"Taws, N.",771,1993.0,1995.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1184,"Sparshott, K.M.",731,1993.0,2002.0,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1192,"Whaite, T.M. | Whaite, J.L.",714,1949.0,1985.0,36.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1208,"Wauchope, J.",695,1923.0,1987.0,64.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Save as CSV file
df_avh_wikidata_cossim.to_csv('data/avhcoll_wikidata_cossim.csv')

In [11]:
# Load MEL Collectors data set
df_melcoll = pd.read_csv('data/mel_collectors.csv')
df_melcoll['full_name'] = df_melcoll['family_name'] + ', ' + df_melcoll['initials']
df_melcoll = df_melcoll[df_melcoll['num_coll'] >= 1000]

df_melcoll

,agent_id,family_name,initials,given_names,num_coll,start_year,end_year,full_name
0,1297,Beauglehole,A.C.,NaN,69198,1929.0,2001.0,"Beauglehole, A.C."
1,14283,Mueller,F.,Ferdinand Jacob Heinrich,25429,1812.0,1895.0,"Mueller, F."
2,19313,Stone,I.G.,NaN,25428,1960.0,1999.0,"Stone, I.G."
3,21883,Willis,J.H.,James,20637,1885.0,1996.0,"Willis, J.H."
4,6458,Filson,R.B.,Rex,15193,1933.0,2000.0,"Filson, R.B."
...,...,...,...,...,...,...,...,...
142,8624,Hartmann,C.H.,Carl,1017,1856.0,1889.0,"Hartmann, C.H."
143,1609,Betche,E.,Ernst,1016,1865.0,1986.0,"Betche, E."
144,8823,Healey,K.,NaN,1014,1953.0,1963.0,"Healey, K."
145,19882,Thies,A.W.,Arthur,1002,1978.0,1998.0,"Thies, A.W."


In [12]:
# Vectorize MEL collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_melcoll['full_name'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_melcoll['full_name'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.03132915496826172


,left_side,right_side,similarity
0,"Beauglehole, A.C.","Beauglehole, A.C.",1.000
1,"Mueller, F.","Mueller, F.v.",0.885
2,"Stone, I.G.","Stone, I.G.",1.000
3,"Willis, J.H.","Willis, J.H.",1.000
4,"Filson, R.B.","Filson, R.B.",1.000


In [13]:
# Join matches data frame to MEL collectors data frame
df_melcoll_matches = pd.merge(df_melcoll, matches_df, left_on='full_name', right_on='left_side', how='left')

# Join Wikidata items
df_melcoll_matches_wikidata = pd.merge(df_melcoll_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_matches_wikidata = pd.merge(df_melcoll_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_matches_wikidata.rename(columns={df_melcoll_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_melcoll_wikidata_cossim = df_melcoll_matches_wikidata[['agent_id', 'family_name', 'initials_x', 'given_names', 
                                                          'num_coll', 'start_year', 'end_year', 'full_name', 
                                                          'left_side', 'right_side', 'similarity', 'dup_count', 
                                                          'item', 'itemLabel', 'surname', 'initials_y', 'canonical_string', 
                                                          'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                          'enc_au_sc_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 
                                                          'orcid_link', 'harv_link', 'ipni_link', 'bloodhound_link', 
                                                          'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_melcoll_wikidata_cossim.sort_values(by=['similarity', 'dup_count', 'num_coll'], ascending=[False, True, False], inplace=True)

df_melcoll_wikidata_cossim

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,agent_id,family_name,initials_x,given_names,num_coll,start_year,end_year,full_name,left_side,right_side,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
0,1297,Beauglehole,A.C.,NaN,69198,1929.0,2001.0,"Beauglehole, A.C.","Beauglehole, A.C.","Beauglehole, A.C.",...,2002.0,NaN,NaN,http://www.wikidata.org/wiki/Q16744919,NaN,NaN,NaN,https://bloodhound-tracker.net/Q16744919,http://www.eoas.info/biogs/P000214b,NaN
2,19313,Stone,I.G.,NaN,25428,1960.0,1999.0,"Stone, I.G.","Stone, I.G.","Stone, I.G.",...,2001.0,NaN,NaN,http://www.wikidata.org/wiki/Q6000720,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/27255-1,NaN,NaN,NaN
5,6458,Filson,R.B.,Rex,15193,1933.0,2000.0,"Filson, R.B.","Filson, R.B.","Filson, R.B.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21339388,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/18803-1,NaN,http://www.eoas.info/biogs/P002347b,NaN
6,6771,Forster,P.I.,Paul Irwin,12226,1955.0,2118.0,"Forster, P.I.","Forster, P.I.","Forster, P.I.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q9057027,NaN,NaN,https://www.ipni.org/a/18907-1,NaN,NaN,NaN
10,20954,Walsh,N.G.,Neville,9122,1971.0,2020.0,"Walsh, N.G.","Walsh, N.G.","Walsh, N.G.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q6041421,NaN,NaN,https://www.ipni.org/a/14171-1,NaN,http://www.eoas.info/biogs/P005329b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,19007,St John,P.R.H.,NaN,4402,1881.0,1999.0,"St John, P.R.H.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,10624,Karunajeewa,N.G.,Nimal,2052,2002.0,2020.0,"Karunajeewa, N.G.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,19624,Tadgell,A.J.,NaN,1425,1886.0,1966.0,"Tadgell, A.J.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,1176,Batianoff,G.N.,NaN,1278,1955.0,2007.0,"Batianoff, G.N.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Save to CSV file
df_melcoll_wikidata_cossim.to_csv('data/melcoll_wikidata_cossim.csv')

In [20]:
# Australian Collectors and illustrators
dtypes = {'coll_index': str, 
          'surname': str,
          'initials': str,
          'canonical_string': str,
          'name': str, 
          'active': str, 
          'comment': str, 
          'info_link': str, 
          'portrait_link': str}
df_chahcoll = pd.read_csv('data/chah_collectors_clean.csv', dtype=dtypes);
df_chahcoll.head()

,Unnamed: 0,coll_index,surname,initials,canonical_string,name,active,comment,info_link,portrait_link
0,0,c00001,ABBOTT,F.,"ABBOTT, F.","ABBOTT, Francis, Jnr",1834–1903,Gardens Superintendent,notes,portrait
1,1,c00002,ABID,M.A.,"ABID, M.A.","ABID, Munir. A., See MUNIR",NaN,NaN,NaN,NaN
2,2,c00003,ABRAHAMS,L.,"ABRAHAMS, L.","ABRAHAMS, L.",fl. 1910,NaN,NaN,NaN
3,3,c00004,ABRAHAMSON,A.,"ABRAHAMSON, A.","ABRAHAMSON, Ada",fl. 1890s,"Amateur seaweed collector, WA",notes,NaN
4,4,c00005,ACKLAND,J.J.,"ACKLAND, J.J.","ACKLAND, Judith Joan",fl. 1960s,Botanist,NaN,NaN


In [23]:
# Vectorize CHAH collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_chahcoll['canonical_string'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_chahcoll['canonical_string'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.25528907775878906


,left_side,right_side,similarity
0,"ABBOTT, F.","Abbott, F.",1.000
1,"ABID, M.A.","Abid, R.",0.764
2,"ABRAHAMS, L.","Abrahams, L.J.",0.918
3,"ABRAHAMSON, A.","Abrahamson, W.",0.928
4,"ACKLAND, J.J.","Ackland, E.",0.815


In [28]:
# Join matches data frame to MEL collectors data frame
df_chahcoll_matches = pd.merge(df_chahcoll, matches_df, left_on='canonical_string', right_on='left_side', how='left')

# Join Wikidata items
df_chahcoll_matches_wikidata = pd.merge(df_chahcoll_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_chahcoll_matches_wikidata = pd.merge(df_chahcoll_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_chahcoll_matches_wikidata.rename(columns={df_chahcoll_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

print(df_chahcoll_matches_wikidata.columns.tolist())

# Remove unwanted columns
df_chahcoll_wikidata_cossim = df_chahcoll_matches_wikidata[['coll_index', 'surname_x', 'initials_x', 'canonical_string_x', 'name', 'active', 'comment', 
                                                          'left_side', 'right_side', 'similarity', 'dup_count', 
                                                          'item', 'itemLabel', 'surname_y', 'initials_y', 'canonical_string_y', 
                                                          'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                          'enc_au_sc_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 
                                                          'orcid_link', 'harv_link', 'ipni_link', 'bloodhound_link', 
                                                          'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_chahcoll_wikidata_cossim.sort_values(by=['similarity', 'dup_count'], ascending=[False, True], inplace=True)

df_chahcoll_wikidata_cossim

['Unnamed: 0', 'coll_index', 'surname_x', 'initials_x', 'canonical_string_x', 'name', 'active', 'comment', 'info_link', 'portrait_link', 'left_side', 'right_side', 'similarity', 'item', 'itemLabel', 'surname_y', 'initials_y', 'canonical_string_y', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 'enc_au_sc_id', 'au_dict_bio', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 'bloodhound_link', 'enc_au_sc_link', 'au_dict_bio_link', ('canonical_string', ''), 'dup_count']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,coll_index,surname_x,initials_x,canonical_string_x,name,active,comment,left_side,right_side,similarity,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
0,c00001,ABBOTT,F.,"ABBOTT, F.","ABBOTT, Francis, Jnr",1834–1903,Gardens Superintendent,"ABBOTT, F.","Abbott, F.",1.0,...,1883.0,NaN,NaN,http://www.wikidata.org/wiki/Q4529517,NaN,NaN,NaN,NaN,http://www.eoas.info/biogs/P000065b,http://adb.anu.edu.au/biography/abbott-francis-4
9,c00010,ADAMS,L.G.(.,"ADAMS, L.G.(.","ADAMS, Laurence George (Laurie)*",1929–2014,"Herbarium botanist, Canberra","ADAMS, L.G.(.","Adams, L.G.",1.0,...,2014.0,NaN,NaN,http://www.wikidata.org/wiki/Q10316562,NaN,NaN,https://www.ipni.org/a/12081-1,NaN,http://www.eoas.info/biogs/P005518b,NaN
11,c00012,ADAMS,M.,"ADAMS, M.","ADAMS, Miss",fl. 1889,Collected on the River Murray,"ADAMS, M.","Adams, M.",1.0,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q22110151,NaN,NaN,NaN,NaN,NaN,NaN
12,c00013,ADAMS,N.,"ADAMS, N.","ADAMS, Nilavan",fl. 2000s,"Botanical artist, ACT","ADAMS, N.","Adams, N.",1.0,...,2007.0,NaN,NaN,http://www.wikidata.org/wiki/Q3297913,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/32209-1,https://bloodhound-tracker.net/Q3297913,NaN,NaN
13,c00014,ADAMS,R.,"ADAMS, R.","ADAMS, Robin*",1953-,Botany Teacher,"ADAMS, R.","Adams, R.",1.0,...,1911.0,NaN,NaN,http://www.wikidata.org/wiki/Q7348773,NaN,NaN,NaN,NaN,NaN,http://adb.anu.edu.au/biography/adams-robert-p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,c03537,YOUNG Andrew fl.1990-2010s Botanist,NaN,YOUNG Andrew fl.1990-2010s Botanist,YOUNG Andrew fl.1990-2010s Botanist,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,c03541,ZACHARIN Robert Fyfe,NaN,ZACHARIN Robert Fyfe,ZACHARIN Robert Fyfe,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4964,c03544,ZECK Emil Hermann,NaN,ZECK Emil Hermann,ZECK Emil Hermann,1891-,?Entomologist,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4965,c03545,ZICH Frank Anthony,NaN,ZICH Frank Anthony,ZICH Frank Anthony *,1968-,Herbarium curator,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Save to CSV file
df_chahcoll_wikidata_cossim.to_csv('data/chahcoll_wikidata_cossim.csv')

In [40]:
# BG Meise collectors
dtypes = {
    'COLL_ID': str, 
    'NAME': str,
    'LNAME': str, 
    'FNAME': str,
    'MNAME': str, 
    'NAME_TO_PRINT': str, 
    'TITLE': str, 
    'POSITION': str, 
    'BIRTH_DT': str, 
    'DEATH_DT': str, 
    'COUNTRY_ONE_LINE': str, 
    'BIRTH_DT': str, 
    'DEATH_DT': str, 
    'COUNTRY_CODE': str
}
df_bgmcoll = pd.read_csv('data/COLLECTORS_28OCT19.TXT', sep='\t', dtype=dtypes)

match_name = []
for i, row in df_bgmcoll.iterrows():
    nstr = str(row['LNAME'])
    fname = str(row['FNAME'])
    mname = str(row['MNAME'])
    if len(fname) > 0:
        words = re.split('[ .]', fname)
        words = [string for string in words if string != ""]
        init = ".".join([x[0] for x in words if len(x) > 0]) + '.'
        nstr = nstr + ', ' + init
        if len(mname) > 0:
            words = re.split('[ .]', mname)
            words = [string for string in words if string != ""]
            init = ".".join([x[0] for x in words if len(x) > 0]) + '.'
            nstr = nstr + init
    match_name.append(nstr)
    
df_bgmcoll['match_name'] = match_name
df_bgmcoll = df_bgmcoll[['COLL_ID', 'NAME', 'LNAME', 'FNAME', 'MNAME', 'match_name']]
df_bgmcoll

,COLL_ID,NAME,LNAME,FNAME,MNAME,match_name
0,L789,L. Leclercq,Leclercq,L.,NaN,"Leclercq, L.n."
1,PLZGE,Pilz G.E.,Pilz,George,E.,"Pilz, G.E."
2,CLIPP,De Clippele,De Clippele,NaN,NaN,"De Clippele, n.n."
3,GRNWL,Greenstock W.,Greenstock,William,NaN,"Greenstock, W.n."
4,R150,Renauld F.,Renauld,F.,NaN,"Renauld, F.n."
...,...,...,...,...,...,...
9911,CARLF,Carlier F.-X.,Carlier,François-Xavier,NaN,"Carlier, F.n."
9912,PANIG,Panigrahi G.,Panigrahi,G.,NaN,"Panigrahi, G.n."
9913,FELFTGI,Feltgen I.,Feltgen,NaN,I.,"Feltgen, n.I."
9914,DEFLD,Defleur D.,Defleur,David,NaN,"Defleur, D.n."


In [41]:
# Vectorize CHAH collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_bgmcoll['match_name'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_bgmcoll['match_name'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.8342916965484619


,left_side,right_side,similarity
0,"Leclercq, L.n.","Leclercq, S.",0.757
1,"Pilz, G.E.","Pilz, G.E.",1.000
2,"Greenstock, W.n.","Bostock, W.",0.546
3,"Renauld, F.n.","Renauld, F.F.G.",0.801
4,"Maula, C.n.","Paula, C.C.d.",0.594


In [44]:
# Join matches data frame to MEL collectors data frame
df_bgmcoll_matches = pd.merge(df_bgmcoll, matches_df, left_on='match_name', right_on='left_side', how='left')

# Join Wikidata items
df_bgmcoll_matches_wikidata = pd.merge(df_bgmcoll_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_bgmcoll_matches_wikidata = pd.merge(df_bgmcoll_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_bgmcoll_matches_wikidata.rename(columns={df_bgmcoll_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_bgmcoll_wikidata_cossim = df_bgmcoll_matches_wikidata[['COLL_ID', 'NAME', 'LNAME', 'FNAME', 'MNAME', 'match_name', 
                                                          'left_side', 'right_side', 'similarity', 'dup_count', 
                                                          'item', 'itemLabel', 'surname', 'initials', 'canonical_string', 
                                                          'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                          'enc_au_sc_id', 'au_dict_bio', 'yob', 'yod', 'wyb', 'wye', 
                                                          'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 
                                                          'bloodhound_link', 'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_bgmcoll_wikidata_cossim.sort_values(by=['similarity', 'dup_count'], ascending=[False, True], inplace=True)

df_bgmcoll_wikidata_cossim

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,COLL_ID,NAME,LNAME,FNAME,MNAME,match_name,left_side,right_side,similarity,dup_count,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
2,PLZGE,Pilz G.E.,Pilz,George,E.,"Pilz, G.E.","Pilz, G.E.","Pilz, G.E.",1.0,1.0,...,2017.0,NaN,NaN,http://www.wikidata.org/wiki/Q21389330,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/24871-1,NaN,NaN,NaN
8,D121,Debeaux O.,Debeaux,Jean,Odon,"Debeaux, J.O.","Debeaux, J.O.","Debeaux, J.O.",1.0,1.0,...,1910.0,NaN,NaN,http://www.wikidata.org/wiki/Q3173773,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/2058-1,NaN,NaN,NaN
21,MLLNG,Miller N.G.,Miller,Norton,George,"Miller, N.G.","Miller, N.G.","Miller, N.G.",1.0,1.0,...,2011.0,NaN,NaN,http://www.wikidata.org/wiki/Q21521514,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23405-1,NaN,NaN,NaN
24,MEIKR,Meikle R.D.,Meikle,Robert,Desmond,"Meikle, R.D.","Meikle, R.D.","Meikle, R.D.",1.0,1.0,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q1309063,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6316-1,NaN,NaN,NaN
46,L952,Laurer J.F.,Laurer,Johann,Friedrich,"Laurer, J.F.","Laurer, J.F.","Laurer, J.F.",1.0,1.0,...,1873.0,NaN,NaN,http://www.wikidata.org/wiki/Q21341241,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/5328-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11345,K050,Kristof L.,Kristof,Lorenz,NaN,"Kristof, L.n.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11353,L108,Laessoe Th.,Laessoe,Th.,NaN,"Laessoe, T.n.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11354,L098,Lemyel,Lemyel,NaN,NaN,"Lemyel, n.n.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11359,VDZOA,Van der Zon A.P.M.,Van der Zon,A.P.M.,NaN,"Van der Zon, A.P.M.n.",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Save as CSV file
df_bgmcoll_wikidata_cossim.to_csv('data/bgmcoll_wikidata_cossim.csv')

### MEL collectors with 500–999 collections

In [59]:
# Load MEL Collectors data set
df_melcoll = pd.read_csv('data/mel_collectors.csv')
# df_melcoll_500['full_name'] = df_melcoll_500['family_name'] + ', ' + df_melcoll_500['initials']

full_name = []
for i, row in df_melcoll.iterrows():
    if row['initials']:
        full_name.append(row['family_name'] + ', ' + str(row['initials']))
    else:
        full_name.append(row['family_name'])
df_melcoll['full_name'] = full_name
    
df_melcoll_1000 = df_melcoll[(df_melcoll['num_coll'] >= 1000)]
df_melcoll_1000.sort_values(['num_coll'], ascending=[False], inplace=True)
df_melcoll_1000.reset_index(drop=True, inplace=True)

df_melcoll_500 = df_melcoll[(df_melcoll['num_coll'] >= 500) & (df_melcoll['num_coll'] < 1000)]
df_melcoll_500.sort_values(['num_coll'], ascending=[False], inplace=True)
df_melcoll_500.reset_index(drop=True, inplace=True)

df_melcoll_100 = df_melcoll[(df_melcoll['num_coll'] >= 100) & (df_melcoll['num_coll'] < 500)]
df_melcoll_100.sort_values(['num_coll'], ascending=[False], inplace=True)
df_melcoll_100.reset_index(drop=True, inplace=True)

print(df_melcoll_1000.shape)
print(df_melcoll_500.shape)
print(df_melcoll_100.shape)

(147, 8)
(138, 8)
(821, 8)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [48]:
# Vectorize MEL collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_melcoll_500['full_name'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_melcoll_500['full_name'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.02211761474609375


,left_side,right_side,similarity
0,"Tenison-Woods, J.E.","Woods, J.",0.588
1,"McDonald, K.R. (1)","McDonald, K.",0.806
2,"Richards, A.F.","Richards, A.M.",0.885
3,"Lau, H.","Lau, S.",0.640
4,"French, C.","French, C.",1.000


In [49]:
# Join matches data frame to MEL collectors data frame
df_melcoll_500_matches = pd.merge(df_melcoll_500, matches_df, left_on='full_name', right_on='left_side', how='left')

# Join Wikidata items
df_melcoll_500_matches_wikidata = pd.merge(df_melcoll_500_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_500_matches_wikidata = pd.merge(df_melcoll_500_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_500_matches_wikidata.rename(columns={df_melcoll_500_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_melcoll_500_wikidata_cossim = df_melcoll_500_matches_wikidata[['agent_id', 'family_name', 'initials_x', 'given_names', 
                                                          'num_coll', 'start_year', 'end_year', 'full_name', 
                                                          'left_side', 'right_side', 'similarity', 'dup_count', 
                                                          'item', 'itemLabel', 'surname', 'initials_y', 'canonical_string', 
                                                          'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                          'enc_au_sc_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 
                                                          'orcid_link', 'harv_link', 'ipni_link', 'bloodhound_link', 
                                                          'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_melcoll_500_wikidata_cossim.sort_values(by=['similarity', 'dup_count', 'num_coll'], ascending=[False, True, False], inplace=True)

df_melcoll_500_wikidata_cossim

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,agent_id,family_name,initials_x,given_names,num_coll,start_year,end_year,full_name,left_side,right_side,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
16,11777,Lepschi,B.J.,Brendan,913,1988.0,2017.0,"Lepschi, B.J.","Lepschi, B.J.","Lepschi, B.J.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q5733654,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/36824-1,NaN,NaN,NaN
17,21369,Weindorfer,G.,Gustav,910,1887.0,1942.0,"Weindorfer, G.","Weindorfer, G.","Weindorfer, G.",...,1932.0,NaN,NaN,http://www.wikidata.org/wiki/Q5621250,NaN,NaN,NaN,NaN,http://www.eoas.info/biogs/P000880b,http://adb.anu.edu.au/biography/weindorfer-gus...
18,4480,Cranfield,R.J.,NaN,906,1965.0,2015.0,"Cranfield, R.J.","Cranfield, R.J.","Cranfield, R.J.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q19263359,NaN,NaN,https://www.ipni.org/a/36616-1,NaN,NaN,NaN
19,6737,Forrest,A.,Alex,893,1871.0,1882.0,"Forrest, A.","Forrest, A.","Forrest, A.",...,1901.0,NaN,NaN,http://www.wikidata.org/wiki/Q706734,NaN,NaN,NaN,NaN,http://www.eoas.info/biogs/P001055b,http://adb.anu.edu.au/biography/forrest-alexan...
24,18945,Spencer,R.D.,Roger,851,1969.0,2002.0,"Spencer, R.D.","Spencer, R.D.","Spencer, R.D.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q7358068,NaN,NaN,https://www.ipni.org/a/14570-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,6143,Errey,E.G.,NaN,518,1976.0,1989.0,"Errey, E.G.","Errey, E.G.","Frey, E.",...,1974.0,NaN,NaN,http://www.wikidata.org/wiki/Q5818904,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/18977-1,NaN,NaN,NaN
169,6143,Errey,E.G.,NaN,518,1976.0,1989.0,"Errey, E.G.","Errey, E.G.","Frey, E.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21518032,NaN,NaN,NaN,NaN,NaN,NaN
93,19410,Strudwick,J.,NaN,686,1986.0,1998.0,"Strudwick, J.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,19411,Strudwick,J.E.,NaN,584,1986.0,2008.0,"Strudwick, J.E.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Save to CSV file
df_melcoll_500_wikidata_cossim.to_csv('data/melcoll_500_wikidata_cossim.csv')

In [54]:
# Vectorize MEL collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_melcoll_100['full_name'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_melcoll_100['full_name'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.06152009963989258


,left_side,right_side,similarity
0,"Walcott, P.","Walcott, J.",0.880
1,"Wilson, F.R.M.","Wilson, F.R.M.",1.000
2,"Fletcher, R.J.","Fletcher, R.A.",0.882
3,"Dittrich, Lieutenant","Dittrich, M.",0.580
4,"Muir, T.","Muir, J.",0.847


In [55]:
# Join matches data frame to MEL collectors data frame
df_melcoll_100_matches = pd.merge(df_melcoll_100, matches_df, left_on='full_name', right_on='left_side', how='left')

# Join Wikidata items
df_melcoll_100_matches_wikidata = pd.merge(df_melcoll_100_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_100_matches_wikidata = pd.merge(df_melcoll_100_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_melcoll_100_matches_wikidata.rename(columns={df_melcoll_100_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_melcoll_100_wikidata_cossim = df_melcoll_100_matches_wikidata[['agent_id', 'family_name', 'initials_x', 'given_names', 
                                                          'num_coll', 'start_year', 'end_year', 'full_name', 
                                                          'left_side', 'right_side', 'similarity', 'dup_count', 
                                                          'item', 'itemLabel', 'surname', 'initials_y', 'canonical_string', 
                                                          'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                          'enc_au_sc_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link', 
                                                          'orcid_link', 'harv_link', 'ipni_link', 'bloodhound_link', 
                                                          'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_melcoll_100_wikidata_cossim.sort_values(by=['similarity', 'dup_count', 'num_coll'], ascending=[False, True, False], inplace=True)

df_melcoll_100_wikidata_cossim

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,agent_id,family_name,initials_x,given_names,num_coll,start_year,end_year,full_name,left_side,right_side,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
1,21939,Wilson,F.R.M.,NaN,496,1882.0,1908.0,"Wilson, F.R.M.","Wilson, F.R.M.","Wilson, F.R.M.",...,1903.0,NaN,NaN,http://www.wikidata.org/wiki/Q5482813,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/11733-1,NaN,http://www.eoas.info/biogs/P005525b,NaN
8,21501,Weymouth,W.A.,NaN,481,1811.0,1921.0,"Weymouth, W.A.","Weymouth, W.A.","Weymouth, W.A.",...,1928.0,NaN,NaN,http://www.wikidata.org/wiki/Q21612552,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/11600-1,NaN,NaN,NaN
9,2039,Boorman,J.L.,John Luke,478,1896.0,1966.0,"Boorman, J.L.","Boorman, J.L.","Boorman, J.L.",...,1919.0,NaN,NaN,http://www.wikidata.org/wiki/Q62088197,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,NaN,http://www.eoas.info/biogs/P005201b,NaN
20,17167,Rogers,R.W.,NaN,464,1960.0,1988.0,"Rogers, R.W.","Rogers, R.W.","Rogers, R.W.",...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21387939,NaN,NaN,https://www.ipni.org/a/25728-1,NaN,NaN,NaN
22,18381,Short,C.W.,Charles Wilkins,457,1812.0,1878.0,"Short, C.W.","Short, C.W.","Short, C.W.",...,1863.0,NaN,NaN,http://www.wikidata.org/wiki/Q15474732,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/9591-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,17389,Rudatis,H.,NaN,122,1908.0,1917.0,"Rudatis, H.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,7529,Gittins,C.H.,NaN,121,1959.0,1975.0,"Gittins, C.H.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,22357,Yugovic,J.,Jeff,110,1974.0,2009.0,"Yugovic, J.",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,5850,Eaves,NaN,NaN,104,1872.0,1927.0,"Eaves, nan",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Save to CSV file
df_melcoll_100_wikidata_cossim.to_csv('data/melcoll_100_wikidata_cossim.csv')

### MEL determiners

In [61]:
df_meldet = pd.read_csv('data/mel_determiners.csv')
df_meldet = df_meldet[df_meldet['num_det'] >= 1000]
df_meldet

,family_name,initials,given_names,name,num_det
0,Jeanes,J.A.,Jeff,"Jeanes, J.A.",34366
1,Stajsic,V.,Val,"Stajsic, V.",27987
2,Beauglehole,A.C.,NaN,"Beauglehole, A.C.",22342
3,Maslin,B.R.,Bruce,"Maslin, B.R.",21100
4,Walsh,N.G.,Neville,"Walsh, N.G.",21075
...,...,...,...,...,...
102,Jessup,L.W.,Laurie,"Jessup, L.W.",1064
103,Hummel,D.A.,David,"Hummel, D.A.",1058
104,Tindale,M.D.,Mary Douglas,"Tindale, M.D.",1014
105,Rule,K.,Kevin,"Rule, K.",1012


In [62]:
# Vectorize MEL collector names (use transform())
tf_idf_matrix_dirty = vectorizer.transform(df_meldet['name'])

# Calculate Cosine Similarity
import time
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)

# Get matches and create data frame
matches_df = get_matches_df(matches, df_meldet['name'], df_wikidata_unique['canonical_string'], top=0)

matches_df.head()

SELFTIMED: 0.019320011138916016


,left_side,right_side,similarity
0,"Jeanes, J.A.","Jeanes, J.A.",1.000
1,"Stajsic, V.","Stajsic, V.",1.000
2,"Beauglehole, A.C.","Beauglehole, A.C.",1.000
3,"Maslin, B.R.","Maslin, B.",0.874
4,"Walsh, N.G.","Walsh, N.G.",1.000


In [64]:
# Join matches data frame to MEL collectors data frame
df_meldet_matches = pd.merge(df_meldet, matches_df, left_on='name', right_on='left_side', how='left')

# Join Wikidata items
df_meldet_matches_wikidata = pd.merge(df_meldet_matches, df_wikidata, left_on='right_side', right_on='canonical_string', how='left')
df_meldet_matches_wikidata = pd.merge(df_meldet_matches_wikidata, df_wikidata_unique, left_on='right_side', right_on='canonical_string', how='left')
df_meldet_matches_wikidata.rename(columns={df_meldet_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)

# Remove unwanted columns
df_meldet_wikidata_cossim = df_meldet_matches_wikidata[['family_name', 'initials_x', 'given_names', 'name', 'num_det', 
                                                        'left_side', 'right_side', 'similarity', 'dup_count', 
                                                        'item', 'itemLabel', 'surname', 'initials_y', 'canonical_string', 
                                                        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bloodhound_id', 
                                                        'enc_au_sc_id', 'au_dict_bio', 'yob', 'yod', 'wyb', 'wye', 
                                                        'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link', 
                                                        'bloodhound_link', 'enc_au_sc_link', 'au_dict_bio_link']]

# Sort by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
df_meldet_wikidata_cossim.sort_values(by=['similarity', 'dup_count', 'num_det'], ascending=[False, True, False], inplace=True)

df_meldet_wikidata_cossim

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,family_name,initials_x,given_names,name,num_det,left_side,right_side,similarity,dup_count,item,...,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bloodhound_link,enc_au_sc_link,au_dict_bio_link
0,Jeanes,J.A.,Jeff,"Jeanes, J.A.",34366,"Jeanes, J.A.","Jeanes, J.A.",1.000,1.0,http://www.wikidata.org/entity/Q21395685,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21395685,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/39451-1,NaN,NaN,NaN
3,Beauglehole,A.C.,NaN,"Beauglehole, A.C.",22342,"Beauglehole, A.C.","Beauglehole, A.C.",1.000,1.0,http://www.wikidata.org/entity/Q16744919,...,2002.0,NaN,NaN,http://www.wikidata.org/wiki/Q16744919,NaN,NaN,NaN,https://bloodhound-tracker.net/Q16744919,http://www.eoas.info/biogs/P000214b,NaN
5,Walsh,N.G.,Neville,"Walsh, N.G.",21075,"Walsh, N.G.","Walsh, N.G.",1.000,1.0,http://www.wikidata.org/entity/Q6041421,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q6041421,NaN,NaN,https://www.ipni.org/a/14171-1,NaN,http://www.eoas.info/biogs/P005329b,NaN
13,Thompson,I.R.,Ian,"Thompson, I.R.",8460,"Thompson, I.R.","Thompson, I.R.",1.000,1.0,http://www.wikidata.org/entity/Q21516217,...,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q21516217,NaN,NaN,https://www.ipni.org/a/37433-1,NaN,NaN,NaN
21,Womersley,H.B.S.,NaN,"Womersley, H.B.S.",5140,"Womersley, H.B.S.","Womersley, H.B.S.",1.000,1.0,http://www.wikidata.org/entity/Q21338083,...,2011.0,NaN,NaN,http://www.wikidata.org/wiki/Q21338083,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/31820-1,NaN,http://www.eoas.info/biogs/P003504b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,Clarke,I.C.,Ian,"Clarke, I.C.",1421,"Clarke, I.C.","Clarke, J.",0.697,4.0,http://www.wikidata.org/entity/Q24205039,...,1895.0,NaN,NaN,http://www.wikidata.org/wiki/Q24205039,NaN,NaN,NaN,NaN,NaN,http://adb.anu.edu.au/biography/clarke-joseph-...
6,Brooker,M.I.H.,NaN,"Brooker, M.I.H.",20371,"Brooker, M.I.H.","Brooker, E.",0.674,1.0,http://www.wikidata.org/entity/Q3048465,...,1948.0,NaN,NaN,http://www.wikidata.org/wiki/Q3048465,NaN,NaN,NaN,NaN,NaN,http://adb.anu.edu.au/biography/brooker-willia...
94,Jolley,H.M.,Helen,"Jolley, H.M.",1486,"Jolley, H.M.","Holley, H.",0.641,1.0,http://www.wikidata.org/entity/Q5901877,...,1827.0,NaN,NaN,http://www.wikidata.org/wiki/Q5901877,NaN,NaN,NaN,NaN,NaN,NaN
118,Vollebergh,P.J.,NaN,"Vollebergh, P.J.",1212,"Vollebergh, P.J.","Bergh, P.J.v.d.",0.558,1.0,http://www.wikidata.org/entity/Q55016623,...,1930.0,NaN,NaN,http://www.wikidata.org/wiki/Q55016623,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# Save to CSV file
df_meldet_wikidata_cossim.to_csv('data/meldet_wikidata_cossim.csv')